In [1]:
%load_ext autoreload
%autoreload 2


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last"

import argparse
from ddpg_torch.ddpg_torch import Agent
import gym
import numpy as np
from lifting_rl.linkage_env_2 import LinkageEnvV2
from lifting_rl.ddpg.utils import OUNoise, NormalizedEnv, Memory

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from livelossplot import PlotLosses

In [4]:
# params = {
#     "N_LINKS": 1,
#     "INIT_STATE": np.array([np.pi / 4, 0], dtype=np.float32),
#     "PARAM_VALS": np.array([9.81, 0.4, 1], dtype=np.float32),
#     "OBS_LOW": np.array([-np.pi, -8 * np.pi], dtype=np.float32),
#     "OBS_HIGH": np.array(
#         [np.pi, 8 * np.pi], dtype=np.float32
#     ),
#     "ACT_LOW": -10,
#     "ACT_HIGH": 10,
#     "TIME_STEP": 0.01,
#     "VIDEO_FPS": 30,
# }

params = {
    "N_LINKS": 5,
    "INIT_STATE": np.array([np.pi / 2, np.pi/2, np.pi/2, -np.pi/4, np.pi/4, 0, 0, 0, 0, 0], dtype=np.float32),
    "PARAM_VALS": np.array([9.81, 0.4, 1, 0.4, 1, 0.6, 1, 0.4, 1, 0.4, 1]),
    "OBS_LOW": np.array([0, np.pi/4, -np.pi/2, -np.pi, -np.pi, -8, -8, -8, -8, -8], dtype=np.float32),
    "OBS_HIGH": np.array(
        [3*np.pi/4, 3*np.pi/2, 3*np.pi/4, np.pi/2, np.pi, 8, 8, 8, 8, 8], dtype=np.float32
    ),
    "ACT_LOW": -100,
    "ACT_HIGH": 100,
    "TIME_STEP": 0.01,
    "VIDEO_FPS": 30,
}


# params = {
#     "N_LINKS": 5,
#     "INIT_STATE": np.array([1.4993862, 1.6887208, 1.5188664, -1.4618884, -1.2326124, 0, 0, 0, 0, 0], dtype=np.float32),
#     "PARAM_VALS": np.array([
#         9.81, 
#         0.4, 5,
#         0.4, 8,
#         0.6, 45, 
#         0.4, 6, 
#         0.4, 5
#     ]),
#     "OBS_LOW": np.array([0, np.pi/4, -np.pi/2, -np.pi, -np.pi, -8, -8, -8, -8, -8], dtype=np.float32),
#     "OBS_HIGH": np.array(
#         [3*np.pi/4, 3*np.pi/2, 3*np.pi/4, np.pi/2, np.pi, 8, 8, 8, 8, 8], dtype=np.float32
#     ),
#     "ACT_LOW": -10,
#     "ACT_HIGH": 10,
#     "TIME_STEP": 0.01,
#     "VIDEO_FPS": 30,
# }


In [5]:
angles_file = "../data/skeleton_angles.csv"

In [6]:
env = LinkageEnvV2(angles_file, params,verbose=0, use_scipy_integration=True) #gym.make("Pendulum-v0") #LinkageEnvV2(angles_file, params, target_pos=np.array([np.pi / 2]) ,verbose=0) #NormalizedEnv( gym.make("Pendulum-v0") )#LinkageEnv(angles_file, params, verbose=0)
env = NormalizedEnv(env)

state_space:  Box(-8.0, 8.0, (10,), float32)
action_space:  Box(-100.0, 100.0, (5,), float32)


In [7]:
# import pickle
# with open('env_5link_simplified.pkl', 'wb') as f:
#     pickle.dump(env, f)

In [8]:
env.action_space

Box(-100.0, 100.0, (5,), float32)

In [9]:
num_states = env.observation_space.shape[0]
num_actions = env.action_space.shape[0]

In [10]:
agent = Agent(
    lr_actor=0.001,
    lr_critic=0.001,
    input_dims=[num_states],
    tau=0.001,
    env=env,
    batch_size=128,
    layer1_size=400,
    layer2_size=400,
    n_actions=num_actions,
)

In [11]:
np.random.seed(0)

In [12]:
liveloss = PlotLosses()

In [13]:
score_history = []

In [ ]:
from IPython.display import clear_output

#liveloss = PlotLosses()
#score_history = []

for i in range(100000):
    done = False
    score = 0
    obs = env.reset()
    agent.noise.reset()
    
    data_q = []
    data_dq = []
    data_tq = []
    data_rewards = []
    data_actions = []
    
    for step in range(500):
#         if i % 20 == 0:
        env.render()
        act = agent.choose_action(obs)
        new_state, reward, done, info = env.step(act)
        agent.remember(obs, act, reward, new_state, int(done))
        agent.learn()
        score += reward
        obs = new_state
        
        #cur_target = env.get_cur_target_pos()
        s = env.state
        data_q.append(s[0])
        data_dq.append(s[1])
        #data_tq.append(cur_target[0])
        data_rewards.append(reward)
        data_actions.append(act[0])
        
        if done:
            break

    
    if i % 20 == 0:
        clear_output()

        plt.plot(data_q)
        plt.plot(data_dq)
    #     plt.plot(data_tq)
        plt.plot()
        plt.show()

        plt.plot(data_rewards)
        plt.plot()
        plt.show()

        plt.plot(data_actions)
        plt.plot()
        plt.show()
    
    score_history.append(score)
    metrics = {
        "score_history": np.mean(score_history[-100:]),
        "score": score
    }
    liveloss.update(metrics)
    liveloss.send()
    print(
        "episode",
        i,
        "score %.2f" % score,
        "100 game average %.2f" % np.mean(score_history[-100:]),
    )
env.close()

/home/peter/miniconda3/envs/rl_env/lib/python3.7/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [37]:
env.close()

In [12]:
a = np.array([1,2,3])

In [14]:
score_history[-100:]

[-2541198.473531222, -272322.8299392415]

In [2]:
import numpy as np
9 * np.pi / 4 % (2 * np.pi)

0.7853981633974483